## Library Widget for Graph Exploration
In this early example, we are demonstrating the ability to explore a graph using the `GraphExplorer` widget. The [InteractiveViewer](Improved_Vis.ipynb) widget is used to explore the graph, and the [GraphExploreNodeSelection](GraphExploreNodeSelection.ipynb) widget is used to populate the initial graph. 

This is an early iteration on the graph exploration capability and is expected to be a bit rough around the edges. Please submit critical bugs to the [issue tracker](https://github.com/jupyrdf/ipyradiant/issues/). 

## Load an RDF graph

In this example, we will use the `ipyradiant` `FileManager`.

In [ ]:
from ipyradiant import FileManager, PathLoader

lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars.ttl"]
lw

## GraphExplorer
The purpose of the `GraphExplorer` widget is to allow users to explore an RDF graph in a way that is easy and intuitive. Exploring RDF graphs helps understand where information is stored, and how things are connected. Greater understanding of the graph's structure will help in downstream tasks such as query development. 

### How To:
In this early version of the `GraphExplorer`, the left-hand panel is used to select an initial set of nodes to populate the graph. 

#### 1. Type Select
From the `Available types:`, choose one or more types that you want to select subject nodes from (e.g. `voc:Droid`, `voc:Film`).

#### 2. Subject Select
From the `Available subjects:` (which should have populated once a type is selected), choose one or more subjects to add to the initial graph.

#### 3. Interactive Viewer
When subjects are selected, nodes should be immediately populated in the interactive viewer. The initially populated nodes are passed through the [RDF2NX](RDF_to_NX.ipynb) process, which means they are LPG nodes with data collapsed from the RDF graph. You can select nodes and the JSON data for the node will be displayed below the main widget viewing area. 

> Note: expanded nodes (i.e. not the initial nodes) are not yet passed through `RDF2NX`. This means that only the initially populated nodes will have all their data from the collapsing process. This will be remedied in a future update to the library. 

##### a. expand a node
When exploring an RDF graph, we can expand upon a node and display connections that are other non-Literal nodes. Once a node is selected, click the `Expand Upon Selected Node` button, which will add all outgoing connections. All node/edges added to the graph are a different color to indicate that they are `temporary`. 

> Note: Only connections to URIRef nodes are included to simplify the representation


##### b. undo last expansion
Exploration is a back-and-forth process. If a node is expanded and it is desired to undo the expansion, the `Undo Last Expansion` will remove all nodes and edges from the last expansion (including nodes locked as `permanent`). 

##### c. make temporary nodes/edges permanent
As nodes are discovered that seem valuable, they can be locked in the graph to prevent removal. Nodes that are `clicked` will have their JSON data visualized, and when clicked <b>again</b> will be locked into the graph. You can confirm this by observing the change in node style. 

> Note: edge style for permanent edges isn't updated until the graph is reset (e.g. upon temporary node removal). This is a known issue with `ipycytoscape` and will be addressed in a future version.

##### d. remove temporary nodes
As the graph gets larger, it may become useful to prune temporary nodes/edges from the viewing area (and underlying graph). The `Remove Temporary Nodes` button will delete all temporary nodes/edges from the graph and reduce the complexity of the visualization. 

In [ ]:
from ipyradiant.visualization.explore import GraphExplorer

In [ ]:
ge = GraphExplorer()
ge.rdf_graph = lw.graph
ge

##### Perform a selection automatically
This allows the notebook to be run completely while still demonstrating the capability (i.e. without human interaction).

In [ ]:
from rdflib import URIRef

# this sets our selection in the widget so that we don't have to click manually
# CAPS vars are used for testing
TSSW_VALUES = (
    URIRef('https://swapi.co/vocabulary/Droid'),
    URIRef('https://swapi.co/vocabulary/Film')
)
ge.node_select.type_select.select_widget.value = TSSW_VALUES

SSSW_VALUES = (
    URIRef('https://swapi.co/resource/film/1'),
    URIRef('https://swapi.co/resource/droid/2'),
    URIRef('https://swapi.co/resource/droid/3')
)
ge.node_select.subject_select.select_widget.value = SSSW_VALUES

In [ ]:
# Select a node automatically (create var for testing)
NODE_TO_SELECT = ge.interactive_viewer.cyto_graph.graph.nodes[0]
ge.interactive_viewer.selected_node = NODE_TO_SELECT